In [1]:
import pandas as pd
from web3 import Web3
from eth_abi import decode

In [2]:
# Set up your Ethereum provider
# rpc_url = 'https://arbitrum.llamarpc.com'
rpc_url = 'https://rpc.ankr.com/arbitrum'

web3_provider = Web3(Web3.HTTPProvider(rpc_url))

In [3]:
connected = web3_provider.is_connected()

In [4]:
# Check if the connection is successful
if connected == True:
    print("Connected")

Connected


In [5]:
# ABI of the Chainlink price feed contract
price_feed_abi = '''
[
    {
        "constant": true,
        "inputs": [],
        "name": "latestRoundData",
        "outputs": [
            {"name": "roundId", "type": "uint80"},
            {"name": "answer", "type": "int256"},
            {"name": "startedAt", "type": "uint256"},
            {"name": "updatedAt", "type": "uint256"},
            {"name": "answeredInRound", "type": "uint80"}
        ],
        "payable": false,
        "stateMutability": "view",
        "type": "function"
    },
    {
        "constant": true,
        "inputs": [
            {"name": "_roundId", "type": "uint80"}
        ],
        "name": "getRoundData",
        "outputs": [
            {"name": "roundId", "type": "uint80"},
            {"name": "answer", "type": "int256"},
            {"name": "startedAt", "type": "uint256"},
            {"name": "updatedAt", "type": "uint256"},
            {"name": "answeredInRound", "type": "uint80"}
        ],
        "payable": false,
        "stateMutability": "view",
        "type": "function"
    }
]
'''


In [6]:
# ABI of the Multicall3 contract
multicall_abi = '''
[
    {
        "constant": true,
        "inputs": [
            {
                "components": [
                    {
                        "name": "target",
                        "type": "address"
                    },
                    {
                        "name": "callData",
                        "type": "bytes"
                    }
                ],
                "name": "calls",
                "type": "tuple[]"
            }
        ],
        "name": "aggregate",
        "outputs": [
            {
                "name": "blockNumber",
                "type": "uint256"
            },
            {
                "name": "returnData",
                "type": "bytes[]"
            }
        ],
        "payable": false,
        "stateMutability": "view",
        "type": "function"
    }
]
'''

In [7]:
multicall_address = '0xcA11bde05977b3631167028862bE2a173976CA11'

In [8]:
# Chainlink ETH/USD price feed contract address
# price_feed_address = '0x639Fe6ab55C921f74e7fac1ee960C0B6293ba612' # Eth
# price_feed_address = '0xb2A824043730FE05F3DA2efaFa1CBbe83fa548D6' # arb
# price_feed_address = '0x6ce185860a4963106506C203335A2910413708e9' # btc
# price_feed_address = '0x02DEd5a7EDDA750E3Eb240b54437a54d57b74dBE' # pepe
# price_feed_address = '0x9C917083fDb403ab5ADbEC26Ee294f6EcAda2720' # uniswap
price_feed_address = '0x66853E19d73c0F9301fe099c324A1E9726953433' # pendle

In [9]:
# Instantiate the contracts
multicall_contract = web3_provider.eth.contract(address=multicall_address, abi=multicall_abi)
price_feed_contract = web3_provider.eth.contract(address=price_feed_address, abi=price_feed_abi)

In [10]:
# Get the latest round data to determine the latest round ID
latest_data = price_feed_contract.functions.latestRoundData().call()
latest_round_id = latest_data[0]
latest_round_id


18446744073709575299

In [11]:
latest_data

[18446744073709575299, 381580000, 1720911963, 1720911963, 18446744073709575299]

In [12]:
# Function to create the calldata for getRoundData
def create_calldata(round_id):
    return price_feed_contract.encodeABI(fn_name='getRoundData', args=[round_id])


In [13]:

# List to store historical data
historical_data = []

# Batch size for multicall
batch_size = 10000

# Fetch historical data in batches, starting from the latest round
for start_round_id in range(latest_round_id, 0, -batch_size):
    calls = []
    for round_id in range(start_round_id, max(start_round_id - batch_size, 0), -1):
        calldata = create_calldata(round_id)
        calls.append((price_feed_address, calldata))
    
    # Prepare the multicall input
    multicall_input = {
        'calls': [(call[0], call[1]) for call in calls]
    }

    # Perform the multicall
    try:
        result = multicall_contract.functions.aggregate(multicall_input['calls']).call()
        block_number, return_data = result[0], result[1]
        
        for data in return_data:
            try:
                decoded_data = decode(['uint80', 'int256', 'uint256', 'uint256', 'uint80'], data)
                round_info = {
                    'roundId': decoded_data[0],
                    'price': decoded_data[1],
                    'startedAt': decoded_data[2],
                    'updatedAt': decoded_data[3],
                    'answeredInRound': decoded_data[4]
                }
                historical_data.append(round_info)
            except Exception as e:
                if 'no data' in str(e):
                    print(f"No data for round {round_id}, exiting.")
                    break
    except Exception as e:
        print(f"Multicall failed for rounds {start_round_id} to {max(start_round_id - batch_size + 1, 0)}: {e}")
        break

    # break

# Convert to DataFrame
df = pd.DataFrame(historical_data)

# Save to CSV
df.to_csv('chainlink_historical_data.csv', index=False)

print("Historical data saved to chainlink_historical_data.csv")

Multicall failed for rounds 18446744073709555299 to 18446744073709545300: ('execution reverted: Multicall3: call failed', '0x08c379a0000000000000000000000000000000000000000000000000000000000000002000000000000000000000000000000000000000000000000000000000000000174d756c746963616c6c333a2063616c6c206661696c6564000000000000000000')
Historical data saved to chainlink_historical_data.csv


In [14]:
df

,roundId,price,startedAt,updatedAt,answeredInRound
0,18446744073709575299,381580000,1720911963,1720911963,18446744073709575299
1,18446744073709575298,379490000,1720911753,1720911753,18446744073709575298
2,18446744073709575297,381607808,1720910822,1720910822,18446744073709575297
3,18446744073709575296,379200000,1720910493,1720910493,18446744073709575296
4,18446744073709575295,376740000,1720910012,1720910012,18446744073709575295
...,...,...,...,...,...
19995,18446744073709555304,107189139,1703171471,1703171471,18446744073709555304
19996,18446744073709555303,107729601,1703169130,1703169130,18446744073709555303
19997,18446744073709555302,108379138,1703168080,1703168080,18446744073709555302
19998,18446744073709555301,108952806,1703167929,1703167929,18446744073709555301
